## Data Loaders

This notebook demonstrates the use of edge neighbor loader in `pyTigerGraph`. The job of a data loader is to pull data from the TigerGraph database. Currently, the following data loaders are provided:
* EdgeLoader, which returns batches of edges.
* VertexLoader, which returns batches of vertices.
* GraphLoader, which returns randomly sampled (probably disconnected) subgraphs in pandas `dataframe`, `PyG` or `DGL` format.
* NeighborLoader, which returns subgraphs using neighbor sampling in `dataframe`, `PyG` or `DGL` format.
* EdgeNeighborLoader, which returns subgraphs using neighbor sampling from edges in `dataframe`, `PyG` or `DGL` format.

Every data loader above can either get all the batches as a HTTP response (default) or stream every batch through Kafka. The former mechanism is good for testing with small graphs and it is fast, but it subjects to a data size limit of 2GB. For large graphs, the HTTP channel will likely fail due to size limit and network connectivity issues. Streaming via Kafka is offered for data robustness and scalability. Also, Kafka excels at multi-consumer use cases, and it is efficient for model search or hyperparameter tuning when there are multiiple consumers of the same data. 

The data loaders support both homogeneous and heterogenous graphs. By default, they load from all vertex and edge types and treat the graph as a homogeneous graph. But they also allow users to specify what vertex and edge types to load from and what attributes to load from each type. This way users will get heterogeneous graph outputs.

**Note**: For the data loaders to work, a UDF (User Defined Function) has to be installed into the TigerGraph database. It only needs to be installed once per database and will work for all data loaders and all graphs. 

### Connection to Database

The `TigerGraphConnection` class represents a connection to the TigerGraph database. Under the hood, it stores the necessary information to communicate with the database. It is able to perform quite a few database tasks. Please see its [documentation](https://docs.tigergraph.com/pytigergraph/current/intro/) for details.

**Note**: Secret instead of username/password is required for TG cloud DBs created after 7/5/2022. Otherwise, you can leave it blank.

In [1]:
from pyTigerGraph import TigerGraphConnection

In [2]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # secret instead of user/pass is required for TG cloud DBs created after 7/5/2022  
)

<span style="color:red">Uncomment cell below and run to get and set token if token authentication is enabled</span>. 
* This is required for all databases on tgcloud.
* `<secret>` is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.
* If you don't know your secret, you can use `secret=conn.createSecret()` to create one.

In [ ]:
#conn.getToken(<secret>)

In [3]:
# Number of vertices for every vertex type
conn.getVertexCount('*')

{'Paper': 2708}

In [4]:
# Number of edges for every type
conn.getEdgeCount()

{'Cite': 10556}

Uncomment below to install the UDF. It only needs to be installed once per database and will work for all data loaders and all graphs. **Note**: installing the UDF will overwrite any existing UDF in the database.

In [5]:
ExprFunctions="https://tg-mlworkbench.s3.us-west-1.amazonaws.com/udf/1.0/ExprFunctions.hpp"  # For enterprise users, please use the link you received.
ExprUtil=""  # For enterprise users, please use the link you received.
conn.installUDF(ExprFunctions, ExprUtil)

ExprFunctions installed succesfully


### Edge Neighbor Loader

`EdgeNeighborLoader` performs neighbor sampling from all edges in the graph in batches in the following manner:

* It chooses a specified number (`batch_size`) of edges as seeds. The number of batches is the total number of edges divided by the batch size. 
  * If you specify the number of batches (`num_batches`) instead, `batch_size` is calculated by dividing the total number of edges by the number of batches.
  * If specify both parameters, `batch_size` takes priority. 
* Starting from the vertices attached to the seed edges, it picks a specified number (`num_neighbors`) of neighbors of each vertex at random.
* It picks the same number of neighbors for each neighbor, and repeats this process until it finished performing a specified number of hops (`num_hops`).
        
This generates one subgraph. As you loop through this data loader, every edge will at some point be chosen as a seed and you will get the subgraph expanded from the seeds. If you want to limit seeds to certain edges, the boolean attribute provided to `filter_by` will be used to indicate which edges can be included as seeds. If you want to load from certain types of vertices and edges, 
use the `dict` input for `v_in_feats`, `v_out_labels`, `v_extra_feats`,
`e_in_feats`, `e_out_labels`, `e_extra_feats` where keys of the dict are vertex 
or edge types to be selected and values are lists of attributes to collect from the
vertex or edge types. 

NOTE: When you initialize the loader on a graph for the first time,
the initialization might take a minute as it installs the corresponding
query to the database. However, the query installation only
needs to be done once, so it will take no time when you initialize the loader
on the same graph again.

    
Args:
* v_in_feats (list or dict, optional):
        Vertex attributes to be used as input features. 
        If it is a list, then the attributes
        in the list from all vertex types will be selected. An error will be thrown if
        certain attribute doesn't exist in all vertex types. If it is a dict, keys of the 
        dict are vertex types to be selected, and values are lists of attributes to be 
        selected for each vertex type.
        Only numeric and boolean attributes are allowed. The type of an attribute 
        is automatically determined from the database schema. Defaults to None.
* v_out_labels (list or dict, optional):
        Vertex attributes to be used as labels for prediction. 
        If it is a list, then the attributes
        in the list from all vertex types will be selected. An error will be thrown if
        certain attribute doesn't exist in all vertex types. If it is a dict, keys of the 
        dict are vertex types to be selected, and values are lists of attributes to be 
        selected for each vertex type.
        Only numeric and boolean attributes are allowed. Defaults to None.
* v_extra_feats (list or dict, optional):
        Other attributes to get such as indicators of train/test data. 
        If it is a list, then the attributes
        in the list from all vertex types will be selected. An error will be thrown if
        certain attribute doesn't exist in all vertex types. If it is a dict, keys of the 
        dict are vertex types to be selected, and values are lists of attributes to be 
        selected for each vertex type. 
        All types of attributes are allowed. Defaults to None.
* e_in_feats (list or dict, optional):
        Edge attributes to be used as input features. 
        If it is a list, then the attributes
        in the list from all edge types will be selected. An error will be thrown if
        certain attribute doesn't exist in all edge types. If it is a dict, keys of the 
        dict are edge types to be selected, and values are lists of attributes to be 
        selected for each edge type.
        Only numeric and boolean attributes are allowed. The type of an attribute
        is automatically determined from the database schema. Defaults to None.
* e_out_labels (list or dict, optional):
        Edge attributes to be used as labels for prediction. 
        If it is a list, then the attributes in the list from all edge types will be 
        selected. An error will be thrown if certain attribute doesn't exist in all 
        edge types. If it is a dict, keys of the dict are edge types to be selected, 
        and values are lists of attributes to be selected for each edge type.
        Only numeric and boolean attributes are allowed. Defaults to None.
* e_extra_feats (list or dict, optional):
        Other edge attributes to get such as indicators of train/test data. 
        If it is a list, then the attributes in the list from all edge types will be 
        selected. An error will be thrown if certain attribute doesn't exist in all 
        edge types. If it is a dict, keys of the dict are edge types to be selected, 
        and values are lists of attributes to be selected for each edge type.
        All types of attributes are allowed. Defaults to None.
* batch_size (int, optional):  
        Number of edges in each batch.  
        Defaults to None.  
* num_batches (int, optional):  
        Number of batches to split the edges.  
        Defaults to 1.  
* num_neighbors (int, optional):
        Number of neighbors to sample for each vertex.
        Defaults to 10.
* num_hops (int, optional):
        Number of hops to traverse when sampling neighbors.
        Defaults to 2.
* shuffle (bool, optional):  
        Whether to shuffle the edges before loading data.  
        Defaults to False.  
* filter_by (str, optional):
        A boolean attribute used to indicate which edges are included. Defaults to None.
* output_format (str, optional):
        Format of the output data of the loader. Only
        "dataframe" is supported. Defaults to "dataframe".
* loader_id (str, optional):
        An identifier of the loader which can be any string. It is
        also used as the Kafka topic name. If `None`, a random string will be generated
        for it. Defaults to None.
* buffer_size (int, optional):
        Number of data batches to prefetch and store in memory. Defaults to 4.
* kafka_address (str, optional):
        Address of the kafka broker. Defaults to None.
* kafka_max_msg_size (int, optional):
        Maximum size of a Kafka message in bytes.
        Defaults to 104857600.
* kafka_num_partitions (int, optional):
        Number of partitions for the topic created by this loader.
        Defaults to 1.
* kafka_replica_factor (int, optional):
        Number of replications for the topic created by this
        loader. Defaults to 1.
* kafka_retention_ms (int, optional):
        Retention time for messages in the topic created by this
        loader in milliseconds. Defaults to 60000.
* kafka_auto_del_topic (bool, optional):
        Whether to delete the Kafka topic once the
        loader finishes pulling data. Defaults to True.
* kafka_address_consumer (str, optional):
        Address of the kafka broker that a consumer
        should use. Defaults to be the same as `kafkaAddress`.
* kafka_address_producer (str, optional):
        Address of the kafka broker that a producer
        should use. Defaults to be the same as `kafkaAddress`.
* timeout (int, optional):
        Timeout value for GSQL queries, in ms. Defaults to 300000.

#### Get subgraphs through http

In [6]:
%%time
neighbor_loader = conn.gds.edgeNeighborLoader(
    num_batches=10,
    num_neighbors = 10,
    num_hops =2,
    v_in_feats = ["x"],
    v_out_labels = ["y"],
    v_extra_feats = ["train_mask", "val_mask", "test_mask"],
    e_in_feats=["time"],
    e_out_labels=[],
    e_extra_feats=["is_train", "is_val"],
    output_format = "PyG",
    shuffle=False,
    filter_by=None
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 213 ms, sys: 42.3 ms, total: 255 ms
Wall time: 45.3 s


In [7]:
%%time
for i, batch in enumerate(neighbor_loader):
    print("----Batch {}----".format(i))
    print(batch)

----Batch 0----
Data(edge_index=[2, 7995], edge_feat=[7995], is_train=[7995], is_val=[7995], is_seed=[7995], x=[2337, 1433], y=[2337], train_mask=[2337], val_mask=[2337], test_mask=[2337])
----Batch 1----
Data(edge_index=[2, 8560], edge_feat=[8560], is_train=[8560], is_val=[8560], is_seed=[8560], x=[2435, 1433], y=[2435], train_mask=[2435], val_mask=[2435], test_mask=[2435])
----Batch 2----
Data(edge_index=[2, 8774], edge_feat=[8774], is_train=[8774], is_val=[8774], is_seed=[8774], x=[2432, 1433], y=[2432], train_mask=[2432], val_mask=[2432], test_mask=[2432])
----Batch 3----
Data(edge_index=[2, 8855], edge_feat=[8855], is_train=[8855], is_val=[8855], is_seed=[8855], x=[2450, 1433], y=[2450], train_mask=[2450], val_mask=[2450], test_mask=[2450])
----Batch 4----
Data(edge_index=[2, 8574], edge_feat=[8574], is_train=[8574], is_val=[8574], is_seed=[8574], x=[2428, 1433], y=[2428], train_mask=[2428], val_mask=[2428], test_mask=[2428])
----Batch 5----
Data(edge_index=[2, 7940], edge_feat=[7

#### For heterogeneous graphs

Since `Cora` is a homogeneous graph, we will connect to a different graph to demostrate the use case of heterogeneous graphs.

In [8]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="hetero",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # secret instead of user/pass is required for TG cloud DBs created after 7/5/2022  
)

# Uncomment below to get and set token if token authentication is enabled. 
#conn.getToken(<secret>) # <secret> is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.

In [9]:
print(conn.gsql("ls"))

---- Graph hetero
Vertex Types:
- VERTEX v0(PRIMARY_ID id INT, x LIST<DOUBLE>, y INT, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX v1(PRIMARY_ID id INT, x LIST<DOUBLE>, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX v2(PRIMARY_ID id INT, x LIST<DOUBLE>, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
Edge Types:
- DIRECTED EDGE v0v0(FROM v0, TO v0, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v1v1(FROM v1, TO v1, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v1v2(FROM v1, TO v2, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v0(FROM v2, TO v0, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v1(FROM v2, TO v1, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v2(FROM v2, TO v2, is_train BOOL, is_val BOOL)

Graphs:
- Graph hetero(v0:v, v1:v, v2:v, v0v0:e, v1v1:e, v1v2:e, v2v0:e,

In [10]:
%%time
neighbor_loader = conn.gds.edgeNeighborLoader(
    v_in_feats={"v0": ["x"],
                "v1": ["x"],
                "v2": ["x"]},
    v_out_labels={"v0": ["y"]},
    v_extra_feats={"v0": ["train_mask", "val_mask", "test_mask"]},
    e_extra_feats={"v0v0": ["is_train"], 
                   "v2v0": ["is_train"],
                   "v1v2": ["is_train"]},
    num_batches=16,
    num_neighbors=10,
    num_hops=2,
    shuffle=False,
    output_format="PyG",
    add_self_loop=False,
    loader_id=None,
    buffer_size=4,
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 26.8 ms, sys: 5.07 ms, total: 31.8 ms
Wall time: 45.9 s


In [11]:
%%time
for i, batch in enumerate(neighbor_loader):
    print("----Batch {}----".format(i))
    print(batch)

----Batch 0----
HeteroData(
  v0={
    x=[76, 77],
    y=[76],
    train_mask=[76],
    val_mask=[76],
    test_mask=[76]
  },
  v1={ x=[46, 57] },
  v2={ x=[100, 48] },
  (v0, v0v0, v0)={
    edge_index=[2, 663],
    is_train=[663],
    is_seed=[663]
  },
  (v1, v1v2, v2)={
    edge_index=[2, 400],
    is_train=[400],
    is_seed=[400]
  },
  (v2, v2v0, v0)={
    edge_index=[2, 884],
    is_train=[884],
    is_seed=[884]
  }
)
----Batch 1----
HeteroData(
  v0={
    x=[76, 77],
    y=[76],
    train_mask=[76],
    val_mask=[76],
    test_mask=[76]
  },
  v1={ x=[47, 57] },
  v2={ x=[99, 48] },
  (v1, v1v2, v2)={
    edge_index=[2, 408],
    is_train=[408],
    is_seed=[408]
  },
  (v2, v2v0, v0)={
    edge_index=[2, 872],
    is_train=[872],
    is_seed=[872]
  },
  (v0, v0v0, v0)={
    edge_index=[2, 655],
    is_train=[655],
    is_seed=[655]
  }
)
----Batch 2----
HeteroData(
  v0={
    x=[76, 77],
    y=[76],
    train_mask=[76],
    val_mask=[76],
    test_mask=[76]
  },
  v1={ x=[

### Streaming through Kafka

**Note**: Kafka streaming is only available for the Enterprise Edition. You need to install the UDF with Kafka support.

In [ ]:
ExprFunctions="./ExprFunctions.hpp"  # For enterprise users, please use the link you received.
ExprUtil="./ExprUtil.hpp"  # For enterprise users, please use the link you received.
conn.installUDF(ExprFunctions, ExprUtil)

#### Configure Kafka
Set up Kafka here. Once configured, the settings will be shared with all newly created data loaders and no need to set up Kafka for each loader. Please see official https://docs.tigergraph.com/pytigergraph/current/gds/gds for detailed settings.

In [ ]:
conn.gds.configureKafka(
    kafka_address="127.0.0.1:9092",
    kafka_security_protocol="SASL_PLAINTEXT",
    kafka_sasl_mechanism="PLAIN",
    kafka_sasl_plain_username="your username",
    kafka_sasl_plain_password="your password"
)

#### Get subgraphs

In [ ]:
%%time
neighbor_loader = conn.gds.EdgeNeighborLoader(
    num_batches=10,
    num_neighbors = 10,
    num_hops =2,
    v_in_feats = ["x"],
    v_out_labels = ["y"],
    v_extra_feats = ["train_mask", "val_mask", "test_mask"],
    e_in_feats=["time"],
    e_out_labels=[],
    e_extra_feats=["is_train", "is_val"],
    output_format = "PyG",
    shuffle=True,
    filter_by=None
)

In [ ]:
%%time
for i, batch in enumerate(neighbor_loader):
    print("----Batch {}----".format(i))
    print(batch)